In [51]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd

# Local imports
from libcomcat.dataframes import (get_detail_data_frame, get_dyfi_data_frame,
                                  get_history_data_frame, get_magnitude_data_frame,
                                  get_pager_data_frame, get_phase_dataframe,
                                  get_summary_data_frame)
from libcomcat.search import search, get_event_by_id
import json
import logging
import time
import warnings
from datetime import datetime, timedelta
from io import StringIO
from xml.dom import minidom

# third party imports
import numpy as np
import pandas as pd
import requests
from esi_utils_geo.compass import get_compass_dir_azimuth
from obspy.geodetics.base import gps2dist_azimuth
from obspy.io.quakeml.core import Unpickler
from scipy.special import erfcinv

# local imports
from libcomcat import search
from libcomcat.exceptions import (
    ParsingError,
    ProductNotFoundError,
    ProductNotSpecifiedError,
)
from libcomcat.utils import HEADERS, TIMEOUT

import warnings

warnings.filterwarnings("ignore")

# Import and combine the datasets

In [2]:
# import all the LA CSV files
la_2000_2023 = pd.read_csv('LA_2000_2023')
la_89_99 = pd.read_csv('LA_1989_1999')
la_79_88 = pd.read_csv('LA_1979_1988')
la_70_78 = pd.read_csv('LA_1970_1978')

In [5]:
# combine all the LA datasets
combine_la = pd.concat([la_2000_2023, la_89_99, la_79_88, la_70_78], axis = 0)
combine_la['City'] = 'Los_Angeles'

In [9]:
# import all the SD CSV files
sd_2000_2023 = pd.read_csv('SD_2000_2023')
sd_89_99 = pd.read_csv('SD_1989_1999')
sd_79_88 = pd.read_csv('SD_1979_1988')
sd_70_78 = pd.read_csv('SD_1970_1978')

<ipython-input-9-2993a714683f>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sd_2000_2023 = pd.read_csv('SD_2000_2023')


In [11]:
combine_sd = pd.concat([sd_2000_2023, sd_89_99, sd_79_88, sd_70_78], axis = 0)
combine_sd['City'] = 'San_Diego'

In [13]:
# import all the SF CSV files
sf_2000_2023 = pd.read_csv('SF_2000_2023')
sf_89_99 = pd.read_csv('SF_1989_1999')
sf_79_88 = pd.read_csv('SF_1979_1988')
sf_70_78 = pd.read_csv('SF_1970_1978')

<ipython-input-13-f9342f260cb5>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sf_2000_2023 = pd.read_csv('SF_2000_2023')


In [14]:
combine_sf = pd.concat([sf_2000_2023, sf_89_99, sf_79_88, sf_70_78], axis = 0)
combine_sf['City'] = 'San_Francisco'

In [15]:
# import all the SJ CSV files
sj_2000_2023 = pd.read_csv('SJ_2000_2023')
sj_89_99 = pd.read_csv('SJ_1989_1999')
sj_79_88 = pd.read_csv('SJ_1979_1988')
sj_70_78 = pd.read_csv('SJ_1970_1978')

<ipython-input-15-3679a8e4e2a8>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sj_2000_2023 = pd.read_csv('SJ_2000_2023')


In [16]:
combine_sj = pd.concat([sj_2000_2023, sj_89_99, sj_79_88, sj_70_78], axis = 0)
combine_sj['City'] = 'San_Jose'

In [17]:
# combine all the cities into one dataset
final_df = pd.concat([combine_la, combine_sd, combine_sf, combine_sj], axis = 0)

In [20]:
final_df.head()

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance,City
0,ci9132068,2000-01-01 12:30:37.190,"13km NE of Yucaipa, California",34.116,-116.940,3.237,1.44,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,32,Los_Angeles
1,ci9132076,2000-01-01 13:56:15.960,"8km SSW of Idyllwild, California",33.677,-116.759,20.746,1.30,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,26,Los_Angeles
2,ci9132077,2000-01-01 14:24:46.730,"5km NE of Running Springs, California",34.238,-117.063,8.960,1.60,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,39,Los_Angeles
3,ci9132083,2000-01-01 15:14:15.740,"9km N of Yucaipa, California",34.112,-117.027,7.307,1.70,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,44,Los_Angeles
4,ci9132121,2000-01-01 20:13:05.180,"1km S of Encino, California",34.149,-118.500,8.910,1.30,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,26,Los_Angeles


# Get Pager_df

In [26]:
pager_filtered_df = final_df[final_df['alert'].isin(['green', 'red', 'yellow'])]
pager_ids = [i for i in pager_filtered_df['id']]
for pager in pager_ids:
    pager_event = get_event_by_id(pager)
    pager_df = get_pager_data_frame(pager_event)
    pager_list.append(pager_df)

new_pager_df = pd.concat(pager_list)

In [28]:
new_pager_df.to_csv('pager_df', index = False)

In [58]:
DYFI_COLUMNS_REPLACE = {
    "Geocoded box": "station",
    "CDI": "intensity",
    "Latitude": "lat",
    "Longitude": "lon",
    "No. of responses": "nresp",
    "Hypocentral distance": "distance",
    "Epicentral distance": "distance",
    "ZIP/Location": "station",
}


In [59]:
def _parse_geojson(bytes_data):
    text_data = bytes_data.decode("utf-8")
    jdict = json.loads(text_data)
    if len(jdict["features"]) == 0:
        return None
    prop_columns = list(jdict["features"][0]["properties"].keys())
    columns = ["lat", "lon"] + prop_columns
    arrays = [[] for col in columns]
    df_dict = dict(zip(columns, arrays))
    for feature in jdict["features"]:
        for column in prop_columns:
            if column == "name":
                prop = feature["properties"][column]
                prop = prop[0 : prop.find("<br>")]
            else:
                prop = feature["properties"][column]

            df_dict[column].append(prop)
        # the geojson defines a box, so let's grab the center point
        lons = [c[0] for c in feature["geometry"]["coordinates"][0]]
        lats = [c[1] for c in feature["geometry"]["coordinates"][0]]
        clon = np.mean(lons)
        clat = np.mean(lats)
        df_dict["lat"].append(clat)
        df_dict["lon"].append(clon)

    df = pd.DataFrame(df_dict)
    df = df.rename(
        index=str, columns={"cdi": "intensity", "dist": "distance", "name": "station"}
    )
    return df


In [60]:
def _parse_text(bytes_geo):
    text_geo = bytes_geo.decode("utf-8")
    lines = text_geo.split("\n")
    columns = lines[0].split(":")[1].split(",")
    columns = [col.strip() for col in columns]
    columns = [col.strip("[") for col in columns]
    columns = [col.strip("]") for col in columns]
    fileio = StringIO(text_geo)
    df = pd.read_csv(fileio, skiprows=1, names=columns)
    df = df.rename(index=str, columns=DYFI_COLUMNS_REPLACE)
    df = df.drop(["Suspect?", "City", "State"], axis=1)
    # df = df[df['nresp'] >= MIN_RESPONSES]
    return df


In [61]:
def get_dyfi_data_frame_2(detail, dyfi_file=None, version="preferred"):
    # ... (existing code)
    if not detail.hasProduct("dyfi"):
        return None
    dyfi = detail.getProducts("dyfi", version=version)[0]
    files = [
        "dyfi_geo_1km.geojson",
        "dyfi_geo_10km.geojson",
        "cdi_geo.txt",
        "cdi_zip.txt",
    ]

    dataframe = None  # Initialize dataframe as None
    if dyfi_file is not None:
        for file in files:
            if file in dyfi.contents:
                data, _ = dyfi.getContentBytes(file)
                if file.endswith("geojson"):
                    dataframe = _parse_geojson(data)
                else:
                    dataframe = _parse_text(data)
                if dataframe is not None and len(dataframe):
                    break  # Data found, exit the loop
            else:
                # Data not found for this file, continue to the next one
                continue
    else:
        for file in files:
            if file in dyfi.contents:
                data, _ = dyfi.getContentBytes(file)
                if file.endswith("geojson"):
                    dataframe = _parse_geojson(data)
                else:
                    dataframe = _parse_text(data)
                if dataframe is not None and len(dataframe):
                    break  # Data found, exit the loop
            else:
                # Data not found for this file, continue to the next one
                continue

    if dataframe is not None and len(dataframe):
        # Add an 'id' column to the DataFrame with the event ID
        dataframe['id'] = detail.id

    return dataframe

In [62]:
# use the ame filtere_df to get the DYFI dataframe
pager_filtered_df = final_df[final_df['alert'].isin(['green', 'red', 'yellow'])]
pager_ids = [i for i in pager_filtered_df['id']]
dyfi_list = []
for pager in pager_ids:
    pager_event = get_event_by_id(pager)
    pager_df = get_dyfi_data_frame_2(pager_event)
    dyfi_list.append(pager_df)
new_dyfi_df = pd.concat(dyfi_list)

In [63]:
new_dyfi_df

,lat,lon,stddev,nresp,station,intensity,distance,id,Standard deviation,cityid
0,36.709540,-121.650847,0.33,1,UTM:(10S 0620 4063 1000),2.2,504,ci9627721,NaN,NaN
1,38.606415,-121.283030,0.33,1,UTM:(10S 0649 4274 1000),3.4,617,ci9627721,NaN,NaN
2,45.238860,-122.827995,0.33,1,UTM:(10T 0513 5009 1000),1.0,1316,ci9627721,NaN,NaN
3,34.441407,-119.737257,0.33,1,UTM:(11S 0248 3814 1000),1.0,257,ci9627721,NaN,NaN
4,35.382470,-119.108288,0.33,1,UTM:(11S 0308 3917 1000),1.0,232,ci9627721,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
180,38.362395,-121.998445,0.33,1,UTM:(10S 0587 4246 1000),2.2,83,nc73895826,NaN,NaN
181,37.803095,-121.937893,0.33,1,UTM:(10S 0593 4184 1000),1.0,132,nc73895826,NaN,NaN
182,37.658165,-121.860600,0.33,1,UTM:(10S 0600 4168 1000),2.7,149,nc73895826,NaN,NaN
183,38.684695,-121.775522,0.33,1,UTM:(10S 0606 4282 1000),2.0,88,nc73895826,NaN,NaN


In [64]:
dyfi_df_csv = new_dyfi_df.to_csv('dyfi_csv_new', index = False)

In [66]:
new_dyfi_df['latitude'] = new_dyfi_df['lat']
new_dyfi_df['longitude'] = new_dyfi_df['lon']
new_dyfi_df_new = new_dyfi_df.drop(columns = ['lat', 'lon', 'Standard deviation', 'cityid', 'station'])
new_dyfi_df_new

,stddev,nresp,intensity,distance,id,latitude,longitude
0,0.33,1,2.2,504,ci9627721,36.709540,-121.650847
1,0.33,1,3.4,617,ci9627721,38.606415,-121.283030
2,0.33,1,1.0,1316,ci9627721,45.238860,-122.827995
3,0.33,1,1.0,257,ci9627721,34.441407,-119.737257
4,0.33,1,1.0,232,ci9627721,35.382470,-119.108288
...,...,...,...,...,...,...,...
180,0.33,1,2.2,83,nc73895826,38.362395,-121.998445
181,0.33,1,1.0,132,nc73895826,37.803095,-121.937893
182,0.33,1,2.7,149,nc73895826,37.658165,-121.860600
183,0.33,1,2.0,88,nc73895826,38.684695,-121.775522


In [68]:
# attempt to join the final df and the DYFI df on the ID column
final_df = pd.merge(final_df, new_dyfi_df_new, on = 'id', how = 'left')
final_df.info()

KeyboardInterrupt: 

In [70]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173360 entries, 0 to 2173359
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   time          object 
 2   location      object 
 3   latitude_x    float64
 4   longitude_x   float64
 5   depth         float64
 6   magnitude     float64
 7   alert         object 
 8   url           object 
 9   eventtype     object 
 10  significance  int64  
 11  City          object 
 12  stddev        float64
 13  nresp         float64
 14  intensity     float64
 15  distance      float64
 16  latitude_y    float64
 17  longitude_y   float64
dtypes: float64(10), int64(1), object(7)
memory usage: 298.5+ MB


In [72]:
final_df['alert'].value_counts()

alert
green     450617
red        35361
yellow      3277
Name: count, dtype: int64

In [73]:
final_df_filtered = final_df[final_df['alert'].isin(['green', 'red', 'yellow'])]

In [74]:
final_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 489255 entries, 4937 to 2019745
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            489255 non-null  object 
 1   time          489255 non-null  object 
 2   location      489255 non-null  object 
 3   latitude_x    489255 non-null  float64
 4   longitude_x   489255 non-null  float64
 5   depth         489255 non-null  float64
 6   magnitude     489255 non-null  float64
 7   alert         489255 non-null  object 
 8   url           489255 non-null  object 
 9   eventtype     489255 non-null  object 
 10  significance  489255 non-null  int64  
 11  City          489255 non-null  object 
 12  stddev        435501 non-null  float64
 13  nresp         489249 non-null  float64
 14  intensity     489249 non-null  float64
 15  distance      489249 non-null  float64
 16  latitude_y    489249 non-null  float64
 17  longitude_y   489249 non-null  float64
dtypes: fl

In [76]:
final_df_filtered.to_csv("Possible_main_df")

In [77]:
final_df_filtered.head()

,id,time,location,latitude_x,longitude_x,depth,magnitude,alert,url,eventtype,significance,City,stddev,nresp,intensity,distance,latitude_y,longitude_y
4937,ci9627721,2001-02-10 21:05:05.780,"6km NNW of Big Bear Lake, CA",34.2895,-116.945833,7.611,4.66,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,814,Los_Angeles,0.33,1.0,2.2,504.0,36.709540,-121.650847
4938,ci9627721,2001-02-10 21:05:05.780,"6km NNW of Big Bear Lake, CA",34.2895,-116.945833,7.611,4.66,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,814,Los_Angeles,0.33,1.0,3.4,617.0,38.606415,-121.283030
4939,ci9627721,2001-02-10 21:05:05.780,"6km NNW of Big Bear Lake, CA",34.2895,-116.945833,7.611,4.66,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,814,Los_Angeles,0.33,1.0,1.0,1316.0,45.238860,-122.827995
4940,ci9627721,2001-02-10 21:05:05.780,"6km NNW of Big Bear Lake, CA",34.2895,-116.945833,7.611,4.66,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,814,Los_Angeles,0.33,1.0,1.0,257.0,34.441407,-119.737257
4941,ci9627721,2001-02-10 21:05:05.780,"6km NNW of Big Bear Lake, CA",34.2895,-116.945833,7.611,4.66,green,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,814,Los_Angeles,0.33,1.0,1.0,232.0,35.382470,-119.108288


In [88]:
grouped_df = final_df_filtered.groupby([
                            # 'City',
                           'latitude_y',
                           'longitude_y'])[['stddev', 'nresp', 'intensity', 'distance', 'depth', 'magnitude','significance']].sum().reset_index()
grouped_df                       

,latitude_y,longitude_y,stddev,nresp,intensity,distance,depth,magnitude,significance
0,-46.144755,167.997580,0.000,1.0,2.7,12129.0,6.98,4.40,808
1,-36.746922,142.226580,0.000,1.0,3.8,13648.0,6.98,4.40,808
2,22.139500,-100.905295,0.311,3.0,3.3,2186.0,8.85,4.17,758
3,22.887830,-109.932430,0.311,3.0,3.7,1514.0,8.85,4.17,758
4,29.085867,-110.974305,0.330,1.0,2.7,739.0,12.96,4.15,715
...,...,...,...,...,...,...,...,...,...
25393,52.533995,6.058512,0.000,1.0,2.0,10214.0,6.98,4.40,808
25394,53.738195,20.522430,0.000,1.0,3.1,11281.0,6.98,4.40,808
25395,53.765095,20.506957,0.000,1.0,3.1,11278.0,6.98,4.40,808
25396,55.674323,12.574912,0.000,1.0,4.8,10535.0,6.98,4.40,808
